# Queries for input and output of search bar 

In [12]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import datetime
plt.rcParams['figure.figsize'] = (15, 5)

with open("copy_cage.json") as config_file:
    config = json.load(config_file)

In [2]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [3]:
search_results = ['https://github.com/ansible/ansible','https://github.com/pulp/pulp-infra-ansible','https://github.com/chaoss/augur','aeshell', 'candlepin']
repo_git_set = []
org_name_set = []

for r in search_results: 
    if r.startswith('http'):
        repo_git_set.append(r)
    else: 
        org_name_set.append(r)

url_query = 'r.repo_git = '
for repo_git in repo_git_set:
    url_query+= '\''
    url_query+=repo_git
    url_query+='\' OR\n\t\tr.repo_git = '
    
url_query = url_query[:-18]

name_query = "rg.rg_name = "
for name in org_name_set:
    name_query+= '\''
    name_query+=name
    name_query+='\' OR\n\t\trg.rg_name = '

name_query = name_query[:-18]
    
print(url_query)
print(name_query)

r.repo_git = 'https://github.com/ansible/ansible' OR
		r.repo_git = 'https://github.com/pulp/pulp-infra-ansible' OR
		r.repo_git = 'https://github.com/chaoss/augur' 
rg.rg_name = 'aeshell' OR
		rg.rg_name = 'candlepin' 


In [4]:
#save
repo_query = salc.sql.text(f"""
        SET SCHEMA 'augur_data';
        SELECT DISTINCT
            r.repo_id,
            r.repo_name
        FROM
            repo r
        JOIN repo_groups rg ON r.repo_group_id = rg.repo_group_id
        WHERE
            {url_query}
        """)


t = engine.execute(repo_query)
results = t.all()
repo_ids = [ row[0] for row in results]
repo_names = [ row[1] for row in results]

print(repo_ids)
print(repo_names)

[27515, 28336, 36113]
['pulp-infra-ansible', 'ansible', 'augur']


In [5]:
org_query = salc.sql.text(f"""
        SET SCHEMA 'augur_data';
        SELECT DISTINCT
            r.repo_id,
            r.repo_name
        FROM
            repo r
        JOIN repo_groups rg ON r.repo_group_id = rg.repo_group_id
        WHERE
            {name_query}
        """)


t = engine.execute(org_query)
results = t.all()
org_repo_ids = [ row[0] for row in results]
org_repo_names = [ row[1] for row in results]

print(org_repo_ids)
print(org_repo_names)

[25453, 25454, 25455, 25456, 25457, 25458, 25459, 25460, 25461, 25462, 25463, 25464, 25465, 25466, 25467, 25468, 25469, 25470, 25471, 25472, 25473, 25474, 25475, 25476, 25477, 25478, 25479, 25480, 25481, 25482, 25483, 34145, 34146, 34147, 34148, 34149, 34150, 34151, 34152, 34153, 34154]
['ansible-role-subman-devel', 'candlepin-deps', 'dependencies', 'python-rhsm', 'candlepin-macros', 'candlepinproject.org', 'candlepin', 'patternfly-react', 'ansible-role-candlepin', 'candlepin-client-java', 'oauth-java-ant', 'zanata-python-client', 'rho', 'caracalla', 'anaconda-addon-development-guide', 'packer', 'initial-setup-port', 'fh-pipeline-library', 'subscription-manager', 'intellij', 'netty-ant', 'pack-utils', 'virt-whom', 'sat5to6', 'virt-who', 'akuma', 'candlepin-jobs', 'openshift-plantuml-cartridge', 'thumbslug', 'python-iniparse', 'openshift-graphviz-cartridge', 'aesh-readline', 'aesh', 'mterm', 'aesh-examples', 'nanook', 'aesh-jvm-tools', 'scalaesh', 'aeshell', 'aesh-extensions', 'examples

In [6]:
total_ids = repo_ids + org_repo_ids
total_names = repo_names + org_repo_names  
print(total_ids)
print(total_names)

[27515, 28336, 36113, 25453, 25454, 25455, 25456, 25457, 25458, 25459, 25460, 25461, 25462, 25463, 25464, 25465, 25466, 25467, 25468, 25469, 25470, 25471, 25472, 25473, 25474, 25475, 25476, 25477, 25478, 25479, 25480, 25481, 25482, 25483, 34145, 34146, 34147, 34148, 34149, 34150, 34151, 34152, 34153, 34154]
['pulp-infra-ansible', 'ansible', 'augur', 'ansible-role-subman-devel', 'candlepin-deps', 'dependencies', 'python-rhsm', 'candlepin-macros', 'candlepinproject.org', 'candlepin', 'patternfly-react', 'ansible-role-candlepin', 'candlepin-client-java', 'oauth-java-ant', 'zanata-python-client', 'rho', 'caracalla', 'anaconda-addon-development-guide', 'packer', 'initial-setup-port', 'fh-pipeline-library', 'subscription-manager', 'intellij', 'netty-ant', 'pack-utils', 'virt-whom', 'sat5to6', 'virt-who', 'akuma', 'candlepin-jobs', 'openshift-plantuml-cartridge', 'thumbslug', 'python-iniparse', 'openshift-graphviz-cartridge', 'aesh-readline', 'aesh', 'mterm', 'aesh-examples', 'nanook', 'aesh-

In [7]:
pr_query = salc.sql.text(f"""
    SET SCHEMA 'augur_data';
        SELECT DISTINCT
            r.repo_git,
            rg.rg_name 
        FROM
            repo r
        JOIN repo_groups rg
        ON rg.repo_group_id = r.repo_group_id
        ORDER BY rg.rg_name
        """)
df_search_bar = pd.read_sql(pr_query, con=engine)

df_search_bar

,repo_git,rg_name
0,https://github.com/3scale/3scale-amp-openshift...,3scale
1,https://github.com/3scale/3scale-api-python,3scale
2,https://github.com/3scale/3scale-api-ruby,3scale
3,https://github.com/3scale/3scale.github.io,3scale
4,https://github.com/3scale/3scale-go-client,3scale
...,...,...
10682,https://github.com/zerodayz/sosreport-operator,zerodayz
10683,https://github.com/zerodayz/talos,zerodayz
10684,https://github.com/zerodayz/tests,zerodayz
10685,https://github.com/zerodayz/web,zerodayz


In [17]:
entries = np.concatenate(( df_search_bar.rg_name.unique() , df_search_bar.repo_git.unique() ), axis=None)
entries = entries.tolist()